# Azure ML Reinforcement Learning Sample

This notebook saves Azure ML workspace for use in the other notebooks.

### Import libraries

In [ ]:
# Azure ML Core imports
import azureml.core
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

### Define Azure ML Subscription and Resource Group details

Define the Azure ML subscription, resource group, region you are using.  Currently, the following regions are suppored: `eastus`, `westeurope`, and `westus2`

In [ ]:
# The Azure subscription you are using
subscription_id='your_subscription_id'
# The resource group for the RL cluster
resource_group='your_resource_group'
# Azure region of the resource group
location='your_region'

### Create Azure ML Worksapce

Create or retrieve the workspace you will use for your RL experiment.

In [ ]:
# Name of your workspace
workspace_name='your_workspace'

ws = Workspace.create(name=workspace_name,
                      subscription_id=subscription_id,
                      resource_group=resource_group,
                      location=location,
                      exist_ok=True)

# Save workspace to local default config
ws.write_config()

# Get the workspace details
ws.get_details()

### Create Virtual Network

If you are using separate compute targets for the Ray head and worker, a virtual network must be created in the resource group.  If you have alraeady created a virtual network in the resource group, you can skip this step.

To do this, you first must install the Azure Networking API.

`pip install --upgrade azure-mgmt-network`

In [ ]:
# If you need to install the Azure Networking SDK, uncomment the following line.
#!pip install --upgrade azure-mgmt-network

In [ ]:
from azure.mgmt.network import NetworkManagementClient
vnet_name ="your_vnet"
subnet_name ="default"
network_client = NetworkManagementClient(ws._auth_object, subscription_id)

async_vnet_creation = network_client.virtual_networks.create_or_update(
    resource_group,
    vnet_name,
    {
        'location': location,
        'address_space': {
            'address_prefixes': ['10.0.0.0/16']
        }
    }
)
async_vnet_creation.wait()

# Create Subnet
async_subnet_creation = network_client.subnets.create_or_update(
    resource_group,
    vnet_name,
    subnet_name,
    {'address_prefix': '10.0.0.0/24'}
)
async_subnet_creation.wait()